In [0]:
import numpy as np

In [0]:
class LBM:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.u = np.zeros((height, width, 2))
        self.fin = np.zeros((height, width, 9))
        self.omega = 1.9 #proportional to viscosity
        self.generateObj()

        u0 = 0.08 #inlet boundary condition for velocity
        self.fin[..., 0] = 4. / 9. * (1 - 1.5 * u0**2)
        self.fin[..., 2] = 1. / 9. * (1 - 1.5 * u0**2)
        self.fin[..., 4] = 1. / 9. * (1 - 1.5 * u0**2)
        self.fin[..., 1] = 1. / 9. * (1 + 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 3] = 1. / 9. * (1 - 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 5] = 1. / 36. * (1 + 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 8] = 1. / 36. * (1 + 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 6] = 1. / 36. * (1 - 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.fin[..., 7] = 1. / 36. * (1 - 3 * u0 + 4.5 * u0**2 - 1.5 * u0**2)
        self.rho = np.sum(self.fin, axis=2)

        self.fin1 = self.fin[0, 0, 1]
        self.fin5 = self.fin[0, 0, 5]
        self.fin8 = self.fin[0, 0, 8]
        self.fin6 = self.fin[0, 0, 6]
        self.fin7 = self.fin[0, 0, 7]
        self.fin3 = self.fin[0, 0, 3]

    def collision(self):
        self.rho = np.sum(self.fin, axis=2)

        self.u[..., 0] = (self.fin[..., 1] + self.fin[..., 5] +
                           self.fin[..., 8] - self.fin[..., 3] -
                           self.fin[..., 7] - self.fin[..., 6]) / self.rho
        self.u[..., 1] = (self.fin[..., 2] + self.fin[..., 5] +
                           self.fin[..., 6] - self.fin[..., 4] -
                           self.fin[..., 7] - self.fin[..., 8]) / self.rho

        u2 = self.u[..., 0]**2 + self.u[..., 1]**2
        uxuy = self.u[..., 0] * self.u[..., 1]
        um32u2 = 1. - 1.5 * u2  #1 minus 3/2 of u**2

        self.fin[..., 0] = self.fin[..., 0] * (
            1. - self.omega) + self.omega * 4. / 9. * self.rho * (um32u2)
        self.fin[..., 1] = self.fin[..., 1] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                3. * self.u[..., 0] + 4.5 * self.u[..., 0]**2 + um32u2)
        self.fin[..., 2] = self.fin[..., 2] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                3. * self.u[..., 1] + 4.5 * self.u[..., 1]**2 + um32u2)
        self.fin[..., 3] = self.fin[..., 3] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                -3. * self.u[..., 0] + 4.5 * self.u[..., 0]**2 + um32u2)
        self.fin[..., 4] = self.fin[..., 4] * (
            1. - self.omega) + self.omega * 1. / 9. * self.rho * (
                -3. * self.u[..., 1] + 4.5 * self.u[..., 1]**2 + um32u2)
        self.fin[..., 5] = self.fin[..., 5] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (self.u[..., 0] + self.u[..., 1]) + 4.5 *
                (u2 + 2. * uxuy) + um32u2)
        self.fin[..., 6] = self.fin[..., 6] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (-self.u[..., 0] + self.u[..., 1]) + 4.5 *
                (u2 - 2. * uxuy) + um32u2)
        self.fin[..., 7] = self.fin[..., 7] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (-self.u[..., 0] - self.u[..., 1]) + 4.5 *
                (u2 + 2. * uxuy) + um32u2)
        self.fin[..., 8] = self.fin[..., 8] * (
            1. - self.omega) + self.omega * 1. / 36. * self.rho * (
                3. * (self.u[..., 0] - self.u[..., 1]) + 4.5 *
                (u2 - 2. * uxuy) + um32u2)

        self.fin[..., 1][:, 0] = self.fin1
        self.fin[..., 3][:, 0] = self.fin3
        self.fin[..., 5][:, 0] = self.fin5
        self.fin[..., 8][:, 0] = self.fin8
        self.fin[..., 6][:, 0] = self.fin6
        self.fin[..., 7][:, 0] = self.fin7

    def streaming(self):
        self.fin[..., 2] = np.roll(self.fin[..., 2], 1, axis=0)
        self.fin[..., 5] = np.roll(self.fin[..., 5], 1, axis=0)
        self.fin[..., 6] = np.roll(self.fin[..., 6], 1, axis=0)

        self.fin[..., 4] = np.roll(self.fin[..., 4], -1, axis=0)
        self.fin[..., 8] = np.roll(self.fin[..., 8], -1, axis=0)
        self.fin[..., 7] = np.roll(self.fin[..., 7], -1, axis=0)

        self.fin[..., 1] = np.roll(self.fin[..., 1], 1, axis=1)
        self.fin[..., 5] = np.roll(self.fin[..., 5], 1, axis=1)
        self.fin[..., 8] = np.roll(self.fin[..., 8], 1, axis=1)

        self.fin[..., 3] = np.roll(self.fin[..., 3], -1, axis=1)
        self.fin[..., 7] = np.roll(self.fin[..., 7], -1, axis=1)
        self.fin[..., 6] = np.roll(self.fin[..., 6], -1, axis=1)

        #bounceback boundary
        self.fin[..., 2][self.objBoundary[..., 2]] = self.fin[..., 4][self.obj]
        self.fin[..., 4][self.objBoundary[..., 4]] = self.fin[..., 2][self.obj]
        self.fin[..., 1][self.objBoundary[..., 1]] = self.fin[..., 3][self.obj]
        self.fin[..., 3][self.objBoundary[..., 3]] = self.fin[..., 1][self.obj]
        self.fin[..., 5][self.objBoundary[..., 5]] = self.fin[..., 7][self.obj]
        self.fin[..., 6][self.objBoundary[..., 6]] = self.fin[..., 8][self.obj]
        self.fin[..., 8][self.objBoundary[..., 8]] = self.fin[..., 6][self.obj]
        self.fin[..., 7][self.objBoundary[..., 7]] = self.fin[..., 5][self.obj]

    def generateObj(self):
        self.obj = np.zeros((self.height, self.width), bool)
        self.objBoundary = np.zeros((self.height, self.width, 9), bool)

        numCircle = np.random.randint(3, 12)
        for _ in range(numCircle):
            radCircle = np.random.randint(5, 10)
            xCenter = np.random.randint(5 + radCircle, .9 * self.width - radCircle)
            yCenter = np.random.randint(5 + radCircle, self.height - radCircle - 5)
            for i in range(self.height):
                for j in range(self.width):
                    if (i - yCenter)**2 + (j - xCenter)**2 <= radCircle**2:
                        self.obj[i, j] = True

        self.obj[0,:] = True
        self.objBoundary[..., 2] = np.roll(self.obj, 1, axis=0)
        self.objBoundary[..., 4] = np.roll(self.obj, -1, axis=0)
        self.objBoundary[..., 1] = np.roll(self.obj, 1, axis=1)
        self.objBoundary[..., 3] = np.roll(self.obj, -1, axis=1)
        self.objBoundary[..., 5] = np.roll(self.objBoundary[..., 2], 1, axis=1)
        self.objBoundary[..., 6] = np.roll(self.objBoundary[..., 2], -1, axis=1)
        self.objBoundary[..., 8] = np.roll(self.objBoundary[..., 4], 1, axis=1)
        self.objBoundary[..., 7] = np.roll(self.objBoundary[..., 4], -1, axis=1)


In [0]:
files_number = 1 #numbers of files to split the dataset
examples_number = 40 #numbers of examples to be generated
simulation_size = 128 #simulation resolution

In [0]:
fluid = LBM(simulation_size, simulation_size)

trainData = np.zeros((examples_number*100, fluid.height, fluid.width, 9))
objData = np.zeros((examples_number*100, fluid.height, fluid.width, 1))

for k in range(files_number):
  x = 0
  for i in range(round(examples_number/files_number)):
    fluid = LBM(simulation_size, simulation_size)
    fluid.generateObj()

    for _ in range(40*120):  #warm up
        fluid.streaming()
        fluid.collision()

    for j in range(100):
        for _ in range(120):
            fluid.streaming()
            fluid.collision()
            
        trainData[x, :, : ,:] = fluid.fin  
        objData[x, :, :, 0] = fluid.obj
        x+=1

    np.save('xtrain',trainData)
    np.save('objtrain', objData)

0
